# Wine Reviews

Comp 562 final project by Chris Burgess, Justin Do, Rhea Gupta, and Kyuyeon Kim.

This project is going to be based around a dataset of win reviews. Our goal is to creat models that are able to make predictions about the kind of grape, score, and area of growth. We are first going to look at cleaning the data and presenting some basic statistics. Then we will move onto how we made the models and lastly how they performed. 

## Importing and Cleaning

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# different environments may not correctly have a working dir. Change this line as appropriate
path_to_data = r"C:\Users\cburg\Documents\wine\data\winemag-data-130k-v2.csv"
df = pd.read_csv(path_to_data)

In [3]:
df.head(3)
print(df.columns)

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')


In [6]:
# Compute a year column for the data

year_reg = r'[12][0-9]{3}'
years = []
for title in df['title']:
    year_result = re.search(year_reg, title)
    if year_result is None:
        years.append('unknown')
    else:
        years.append(year_result.group(0))

df['year'] = years

df.head(3)

2019


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
